In [5]:
from typing import List
import matplotlib.pyplot as plt
import random
import copy
import numpy as np
class Position:
    def __init__(self, x,y):
        self.x = x
        self.y=y
class Elf(Position):
    def step(self,direction:str):
        self.prev = (self.x, self.y)
        if direction =="up":
            self.x += -1
        elif direction =="down":
            self.x += 1
        elif direction =="left":
            self.y += -1
        elif direction =="right":
            self.y += 1
    def backup(self):
        self.x, self.y = self.prev
class Environment:
    def __init__(self, entrance=None,file_path="day24/input.txt", elf =None):
        with open(file_path,"r") as file:
            lines = file.readlines()
        raw_data = [line.strip() for line in lines]
        raw_temp =[]
        for line in raw_data:
            temp = []
            for element in line:
                temp.append(element)
            raw_temp.append(temp)
        raw_data = np.array(raw_temp)
        inner_data = raw_data[1:raw_data.shape[0]-1,1:raw_data.shape[1]-1]
        outer_data = np.where(raw_data=="#", 1, 0 )
        if not elf:
            self.elf = Elf(0, 1)
            outer_data[0,1] = 0

        if not entrance:
            self.entrance = outer_data.shape[0]-1,outer_data.shape[1]-2

        else:
            self.entrance = entrance
        outer_data[self.entrance[0],self.entrance[1]] =-1

        self.outer_data = outer_data
        self.left_data = np.where(inner_data=="<", 2, 0 )
        self.right_data = np.where(inner_data==">",4, 0)
        self.up_data = np.where(inner_data=="^",3, 0)
        self.down_data = np.where(inner_data=="v",5, 0)
        self.time =0
        self.game_status = "active"
    def check_status(self):
        data = self.get_data()
        if (self.elf.x, self.elf.y) == self.entrance:
            return 1
        if self.elf.x >= data.shape[0]-1 or self.elf.y >= data.shape[1]-1 or self.elf.x <0 or self.elf.y <0:
            return -1
        value = data[self.elf.x][self.elf.y]
        if value == -1:
            return 1
        elif value ==0:
            return 0
        else: return -1
        
    def get_data(self):
        re_inner_data = self.left_data+self.right_data+self.up_data + self.down_data
        re_inner_data = np.pad(re_inner_data,1,constant_values=0)
        return re_inner_data + self.outer_data
    def progress_env(self, n_steps, reset_state=False):
        # print("progress received n_steps ", n_steps, "reset_state ", reset_state, " time ", self.time )


        if not reset_state:
            self.time += n_steps
            roll_steps = n_steps
        else:
            roll_steps = n_steps- self.time
            self.time = n_steps
        self.left_data =np.roll(self.left_data,-roll_steps,axis =1)
        self.right_data =np.roll(self.right_data,roll_steps,axis =1)
        self.up_data =np.roll(self.up_data,-roll_steps,axis =0)
        self.down_data =np.roll(self.down_data,roll_steps,axis =0)
        # assert self.check_status() >=0, str(self.elf.x) + " " + str(self.elf.y) + " " + str(hash(str(env.get_data()))) + " time " + str(self.time)

    def backup(self):

        self.elf.backup()
        # print("call progress_env at backup")
        self.progress_env(-1)

    def step(self,direction) -> int:
        # print("call progress at step")
        self.progress_env(1)
        self.elf.step(direction)
        if self.check_status() <0:
            self.game_status = "over"
            return -1
        elif self.check_status() >0:
            self.game_status = "done"
            return 1
        else: 
            self.game_status = "active"
            return 0
    def printout(self):
        data = self.get_data()
        i=0
        for row in data:
            j=0
            print()
            for item in row:
                if (i,j) == (self.elf.x,self.elf.y):
                    if item!=0:
                        print("X", end="")
                    else:
                        print("E", end="")
                else:
                    if item==1:
                        print('#', end="")
                    elif item==0 or item ==-1:
                        print(".", end="")
                    elif item == 2:
                        print("<", end="")
                    elif item == 4:
                        print(">", end="")
                    elif item == 3:
                        print("^", end="")
                    elif item == 5:
                        print("v", end="")
                    else:
                        print("&", end="")
                j += 1
            i += 1

    def clone(self):
        env = copy.deepcopy(self)
        return env


Dijkstras implementation

In [207]:
env = Environment()
data = env.get_data()

In [160]:
env = Environment(file_path="day24/example_input.txt")
data = env.get_data()
env.printout()


#E######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#

In [6]:
import sys
sys.setrecursionlimit(100000)
starting_position=(0,1)
env = Environment(file_path="day24/input.txt")
data = env.get_data()
entrance=(data.shape[0]-1, data.shape[1]-2 )

state_map = {}
dead_nodes = []
MAX_NODES = 5800000
MAX_DEPTH = [18]
MAX_WAIT = 3
STOP_FLAG = [False]
nodes_count =[0]


nodes = set([(i,j) for j in range(1, data.shape[1]-1)for i in range(1, data.shape[0]-1) ])
##adding entrance and starting nodes
nodes.add(starting_position)
nodes.add(entrance)

def test_action(action, env):
        reward = env.step(action)
        # print("test action ", action, "elf: ", env.elf.x, " ", env.elf.y, " time ", env.time)
        # print(hash(str(env.get_data())))
        status = env.check_status()
        x,y, time = env.elf.x, env.elf.y, env.time
        env_state = hash(str(env.get_data()))
        env.backup()
        # print("reward ", reward, "status",status, "elf: ", x, " ", y, " time ", time)

        return reward >= 0, (x,y,env_state, time)
def expand(node, env,time, actions, max_depth):
    # print("input node ", node, "time ", time, "actions ", actions)
    if STOP_FLAG[0]: return
    if node in nodes:
        nodes.remove(node)
    if len(nodes) ==0:
        return
    # print("len nodes ", len(nodes))
    # print("node count", nodes_count[0])
    env_hash = hash(str(env.get_data()))
    node_id = (node[0], node[1], env_hash)
    if time >= max_depth:
        # print("max_depth reached")

        return
    if node_id in dead_nodes:
        # print("deadnode reached")
        return
    if nodes_count[0] >= MAX_NODES:
        print("max node reached")
        return
    env.elf.x, env.elf.y = node[0],node[1] #update elf position for environment
    env.progress_env(time,reset_state=True) #update the environment progress
    output=[]
    ok_actions =[]
    time_before_action = env.time
    for action in ["up","down", "left", "right"]:
        result = test_action(action, env)
        if result[0]:
            output.append((result[1][0],result[1][1],result[1][2],result[1][3],actions+[action]))
            ok_actions.append(action)
    waits=[]
    wait_count =0
    actions_to_wait_for = set(["up","down", "left", "right"]) 

    if len(actions_to_wait_for) >0:
        for _ in range(MAX_WAIT):

            env.step("wait")
            if env.check_status() >=0:
                waits.append("wait")
                wait_count +=1
            else:
                env.backup()
                break
            for action in actions_to_wait_for:
                result = test_action(action, env)
                if result[0]:
                    output.append((result[1][0],result[1][1],result[1][2],result[1][3],actions+waits+[action]))
        # print(" call progress_env at wait call")
        env.progress_env(-wait_count) #reverse environment back
    if len(output) >0: 
        nodes_count[0] += 1
    else:
        dead_nodes.append(node_id)
    
    for expanded_node in output:
        state = (expanded_node[0], expanded_node[1],expanded_node[2])
        time = expanded_node[3]
        actions = expanded_node[4]
        if (expanded_node[0], expanded_node[1]) == entrance:
            print("solved, the shortest time is ", time )
            STOP_FLAG[0] = True
            return True
            # if time < MAX_DEPTH[0]:
            #     print("reached entrance, resetting max depth to ", time)
            #     MAX_DEPTH[0]= time
            
        existing_state= state_map.get(state,"")
        if existing_state != "":
            if existing_state[0] > time:
                state_map[state] = (time, actions)
        else:
            state_map[state] = (time, actions)
        expand((expanded_node[0], expanded_node[1]), env, time,actions, max_depth)
for max_depth in range(10, 50):
    print("try with max_depth ", max_depth)
    if expand(starting_position, env,0,[], max_depth) is True:
        break

try with max_depth  10
try with max_depth  11
try with max_depth  12
try with max_depth  13
try with max_depth  14
try with max_depth  15
try with max_depth  16
try with max_depth  17
try with max_depth  18
try with max_depth  19
